# Riemann Zeros Toolkit for GIFT Research

**Purpose**: Download, cache, and analyze Riemann zeta zeros for GIFT correspondence research.

**Features**:
- Bulk download from Odlyzko tables (2M+ zeros)
- Local caching for fast reload
- CuPy GPU acceleration for analysis
- mpmath integration for high-precision computation
- GIFT correspondence validation tools

**Requirements**: Colab Pro+ with A100 recommended for large-scale analysis.

In [ ]:
# Install dependencies (run once)
!pip install -q mpmath requests tqdm

In [ ]:
import os
import json
import requests
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from typing import Optional, Tuple, List
import warnings

# GPU support (optional)
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print(f"✓ CuPy available - GPU: {cp.cuda.runtime.getDeviceProperties(0)['name'].decode()}")
except ImportError:
    cp = np  # Fallback to NumPy
    GPU_AVAILABLE = False
    print("⚠ CuPy not available, using NumPy (CPU)")

# High precision (optional)
try:
    from mpmath import mp, zetazero, zeta
    mp.dps = 50  # 50 decimal places default
    MPMATH_AVAILABLE = True
    print(f"✓ mpmath available - precision: {mp.dps} digits")
except ImportError:
    MPMATH_AVAILABLE = False
    print("⚠ mpmath not available, high-precision computation disabled")

## 1. Data Sources Configuration

In [ ]:
# Odlyzko data sources
ODLYZKO_BASE = "https://www-users.cse.umn.edu/~odlyzko/zeta_tables"

ODLYZKO_FILES = {
    "zeros1": {
        "url": f"{ODLYZKO_BASE}/zeros1",
        "count": 100000,
        "start_index": 1,
        "precision": "9 decimal places",
        "description": "First 100,000 zeros"
    },
    "zeros2": {
        "url": f"{ODLYZKO_BASE}/zeros2",
        "count": 1000,
        "start_index": 10**12 + 1,
        "precision": "9 decimal places",
        "description": "Zeros 10^12+1 to 10^12+1000"
    },
    "zeros3": {
        "url": f"{ODLYZKO_BASE}/zeros3",
        "count": 10001,
        "start_index": 10**21 + 1,
        "precision": "12 decimal places",
        "description": "Zeros around 10^21"
    },
    "zeros4": {
        "url": f"{ODLYZKO_BASE}/zeros4",
        "count": 10001,
        "start_index": 10**22 - 10**7 + 1,
        "precision": "9 decimal places",
        "description": "Zeros around 10^22"
    },
    "zeros5": {
        "url": f"{ODLYZKO_BASE}/zeros5",
        "count": 300000,
        "start_index": 10**22 + 1,
        "precision": "8 decimal places",
        "description": "300,000 zeros starting at 10^22"
    },
    "zeros6": {
        "url": f"{ODLYZKO_BASE}/zeros6.gz",
        "count": 2001052,
        "start_index": 1,
        "precision": "8 decimal places",
        "description": "First 2,001,052 zeros (gzipped)",
        "compressed": True
    }
}

# High precision sources
HIGH_PRECISION = {
    "plouffe_100": {
        "url": "http://www.plouffe.fr/simon/constants/zeta100.html",
        "count": 100,
        "precision": "1000+ decimal places",
        "description": "First 100 zeros, ultra-high precision"
    }
}

# GIFT constants for validation
GIFT_CONSTANTS = {
    "dim_G2": 14,
    "b2": 21,
    "b3": 77,
    "H_star": 99,
    "dim_E8": 248,
    "dim_K7": 7,
    "rank_E8": 8,
    "Weyl": 5,
    "dim_J3O": 27,
    "F7": 13,  # 7th Fibonacci
    "kappa_T_inv": 61
}

print(f"Configured {len(ODLYZKO_FILES)} Odlyzko sources")
print(f"Total zeros available: {sum(f['count'] for f in ODLYZKO_FILES.values()):,}")

## 2. Download and Cache Manager

In [ ]:
class RiemannZeroCache:
    """Manages downloading and caching of Riemann zero tables."""
    
    def __init__(self, cache_dir: str = "./riemann_cache"):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        self.metadata_file = self.cache_dir / "metadata.json"
        self._load_metadata()
        
    def _load_metadata(self):
        if self.metadata_file.exists():
            with open(self.metadata_file) as f:
                self.metadata = json.load(f)
        else:
            self.metadata = {"downloaded": {}, "computed": {}}
    
    def _save_metadata(self):
        with open(self.metadata_file, 'w') as f:
            # Convert numpy types to Python types for JSON
            json.dump(self.metadata, f, indent=2, default=lambda x: float(x) if hasattr(x, 'item') else x)
    
    def download_odlyzko(self, name: str, force: bool = False) -> np.ndarray:
        """Download an Odlyzko table and cache locally."""
        if name not in ODLYZKO_FILES:
            raise ValueError(f"Unknown file: {name}. Available: {list(ODLYZKO_FILES.keys())}")
        
        info = ODLYZKO_FILES[name]
        cache_file = self.cache_dir / f"{name}.npy"
        
        # Check cache
        if cache_file.exists() and not force:
            print(f"Loading {name} from cache...")
            return np.load(cache_file)
        
        # Download
        print(f"Downloading {info['description']}...")
        response = requests.get(info['url'], stream=True)
        response.raise_for_status()
        
        # Parse content
        if info.get('compressed'):
            import gzip
            content = gzip.decompress(response.content).decode('utf-8')
        else:
            content = response.text
        
        # Parse zeros (one per line)
        zeros = []
        for line in tqdm(content.strip().split('\n'), desc="Parsing"):
            line = line.strip()
            if line and not line.startswith('#'):
                try:
                    zeros.append(float(line))
                except ValueError:
                    continue
        
        zeros = np.array(zeros, dtype=np.float64)
        
        # Cache
        np.save(cache_file, zeros)
        self.metadata['downloaded'][name] = {
            'count': len(zeros),
            'min': float(zeros.min()),
            'max': float(zeros.max()),
            'start_index': info['start_index']
        }
        self._save_metadata()
        
        print(f"✓ Cached {len(zeros):,} zeros to {cache_file}")
        return zeros
    
    def download_all(self, force: bool = False) -> dict:
        """Download all Odlyzko tables."""
        results = {}
        for name in ODLYZKO_FILES:
            try:
                results[name] = self.download_odlyzko(name, force=force)
            except Exception as e:
                print(f"⚠ Failed to download {name}: {e}")
        return results
    
    def get_zeros(self, start: int = 1, end: int = 100000) -> np.ndarray:
        """Get zeros by index range (1-indexed)."""
        # Load zeros1 (first 100k) or zeros6 (first 2M)
        if end <= 100000:
            zeros = self.download_odlyzko("zeros1")
        else:
            zeros = self.download_odlyzko("zeros6")
        
        # Convert to 0-indexed
        return zeros[start-1:end]
    
    def status(self):
        """Print cache status."""
        print("\n=== Riemann Zero Cache Status ===")
        total = 0
        for name, info in self.metadata.get('downloaded', {}).items():
            print(f"  {name}: {info['count']:,} zeros (γ ∈ [{info['min']:.2f}, {info['max']:.2f}])")
            total += info['count']
        print(f"  Total cached: {total:,} zeros")
        print(f"  Cache dir: {self.cache_dir.absolute()}")

# Initialize cache
cache = RiemannZeroCache()
cache.status()

## 3. Download Data (Run Once)

In [ ]:
# Download the main dataset (first 100,000 zeros) - quick
zeros_100k = cache.download_odlyzko("zeros1")
print(f"\nFirst 10 zeros: {zeros_100k[:10]}")
print(f"γ₁ = {zeros_100k[0]:.10f} (GIFT: dim(G₂) = 14, deviation: {abs(zeros_100k[0] - 14)/14*100:.2f}%)")

In [ ]:
# Optional: Download full 2M dataset (takes longer, ~40MB)
# Uncomment to run:
# zeros_2M = cache.download_odlyzko("zeros6")
# print(f"Loaded {len(zeros_2M):,} zeros")

## 4. GPU-Accelerated Analysis with CuPy

In [ ]:
class GIFTRiemannAnalyzer:
    """GPU-accelerated analysis of GIFT-Riemann correspondences."""
    
    def __init__(self, zeros: np.ndarray, use_gpu: bool = True):
        self.use_gpu = use_gpu and GPU_AVAILABLE
        self.xp = cp if self.use_gpu else np
        
        # Transfer to GPU if available
        if self.use_gpu:
            self.zeros = cp.asarray(zeros)
            print(f"✓ Loaded {len(zeros):,} zeros on GPU")
        else:
            self.zeros = zeros
            print(f"✓ Loaded {len(zeros):,} zeros on CPU")
    
    def find_near_integer(self, target: int, tolerance: float = 0.5) -> dict:
        """Find zeros nearest to a target integer."""
        xp = self.xp
        diff = xp.abs(self.zeros - target)
        mask = diff < tolerance
        
        if not xp.any(mask):
            # Find closest anyway
            idx = int(xp.argmin(diff))
            return {
                'target': target,
                'nearest_index': idx + 1,  # 1-indexed
                'value': float(self.zeros[idx]),
                'deviation': float(diff[idx]),
                'deviation_pct': float(diff[idx] / target * 100)
            }
        
        indices = xp.where(mask)[0]
        if self.use_gpu:
            indices = indices.get()
            values = self.zeros[mask].get()
            deviations = diff[mask].get()
        else:
            values = self.zeros[mask]
            deviations = diff[mask]
        
        best_idx = int(indices[np.argmin(deviations)])
        return {
            'target': target,
            'matches': len(indices),
            'nearest_index': best_idx + 1,
            'value': float(self.zeros[best_idx]),
            'deviation': float(np.min(deviations)),
            'deviation_pct': float(np.min(deviations) / target * 100)
        }
    
    def validate_gift_correspondences(self) -> dict:
        """Validate all GIFT-Riemann correspondences."""
        results = {}
        
        # Known correspondences from research
        correspondences = [
            (1, 14, "dim(G₂)"),
            (2, 21, "b₂"),
            (20, 77, "b₃"),
            (29, 99, "H*"),
            (107, 248, "dim(E₈)"),
        ]
        
        print("\n=== GIFT-Riemann Correspondences ===")
        print(f"{'Index':<8} {'γₙ':<12} {'Target':<8} {'Constant':<12} {'Dev %':<10}")
        print("-" * 55)
        
        for idx, target, name in correspondences:
            if idx <= len(self.zeros):
                gamma = float(self.zeros[idx - 1])  # 0-indexed
                dev_pct = abs(gamma - target) / target * 100
                results[name] = {
                    'index': idx,
                    'gamma': gamma,
                    'target': target,
                    'deviation_pct': dev_pct
                }
                print(f"γ_{idx:<5} {gamma:<12.6f} {target:<8} {name:<12} {dev_pct:<10.4f}")
        
        mean_dev = np.mean([r['deviation_pct'] for r in results.values()])
        print(f"\nMean deviation: {mean_dev:.4f}%")
        results['mean_deviation'] = mean_dev
        
        return results
    
    def test_pell_equation(self) -> dict:
        """Test the modified Pell equation: γ₂₉² - 49γ₁² + γ₂ + 1 ≈ 0"""
        g1 = float(self.zeros[0])   # γ₁
        g2 = float(self.zeros[1])   # γ₂
        g29 = float(self.zeros[28]) # γ₂₉
        
        # Modified Pell: γ₂₉² - 49γ₁² + γ₂ + 1 ≈ 0
        pell_value = g29**2 - 49 * g1**2 + g2 + 1
        
        print("\n=== Modified Pell Equation ===")
        print(f"γ₁  = {g1:.10f}")
        print(f"γ₂  = {g2:.10f}")
        print(f"γ₂₉ = {g29:.10f}")
        print(f"\nγ₂₉² - 49γ₁² + γ₂ + 1 = {pell_value:.10f}")
        print(f"Expected: 0")
        print(f"Relative error: {abs(pell_value) / g29**2 * 100:.6f}%")
        
        return {
            'gamma_1': g1,
            'gamma_2': g2,
            'gamma_29': g29,
            'pell_value': pell_value,
            'relative_error_pct': abs(pell_value) / g29**2 * 100
        }
    
    def test_recurrence(self, n_test: int = 1000) -> dict:
        """Test the GIFT recurrence: γₙ ≈ a₅γₙ₋₅ + a₈γₙ₋₈ + a₁₃γₙ₋₁₃ + a₂₇γₙ₋₂₇ + c"""
        xp = self.xp
        
        # Need at least 28 zeros for lag-27
        if len(self.zeros) < 28 + n_test:
            n_test = len(self.zeros) - 28
        
        # GIFT lags: Weyl=5, rank(E₈)=8, F₇=13, dim(J₃(O))=27
        lags = [5, 8, 13, 27]
        
        # Build design matrix for linear regression
        # γₙ = a₅γₙ₋₅ + a₈γₙ₋₈ + a₁₃γₙ₋₁₃ + a₂₇γₙ₋₂₇ + c
        start = max(lags)
        end = start + n_test
        
        if self.use_gpu:
            zeros_cpu = self.zeros.get()
        else:
            zeros_cpu = self.zeros
        
        # Design matrix
        X = np.column_stack([
            zeros_cpu[start - lag:end - lag] for lag in lags
        ] + [np.ones(n_test)])  # constant term
        
        y = zeros_cpu[start:end]
        
        # Least squares fit
        coeffs, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)
        
        # Predictions and errors
        y_pred = X @ coeffs
        errors = np.abs(y - y_pred)
        rel_errors = errors / y * 100
        
        print(f"\n=== Recurrence Analysis (n={n_test:,}) ===")
        print(f"\nFitted coefficients:")
        for lag, coef in zip(lags, coeffs[:-1]):
            print(f"  a_{lag} = {coef:.6f}")
        print(f"  c   = {coeffs[-1]:.6f}")
        print(f"\nError statistics:")
        print(f"  Mean relative error: {np.mean(rel_errors):.4f}%")
        print(f"  Max relative error:  {np.max(rel_errors):.4f}%")
        print(f"  Std relative error:  {np.std(rel_errors):.4f}%")
        
        return {
            'lags': lags,
            'coefficients': {f'a_{lag}': float(c) for lag, c in zip(lags, coeffs[:-1])},
            'constant': float(coeffs[-1]),
            'mean_rel_error_pct': float(np.mean(rel_errors)),
            'max_rel_error_pct': float(np.max(rel_errors)),
            'std_rel_error_pct': float(np.std(rel_errors)),
            'n_samples': n_test
        }
    
    def scan_for_gift_constants(self, tolerance_pct: float = 1.0) -> list:
        """Scan all zeros for proximity to GIFT constants."""
        results = []
        
        for name, value in GIFT_CONSTANTS.items():
            match = self.find_near_integer(value, tolerance=value * tolerance_pct / 100)
            match['gift_constant'] = name
            results.append(match)
        
        print(f"\n=== Scan for GIFT Constants (tolerance: {tolerance_pct}%) ===")
        print(f"{'Constant':<15} {'Value':<8} {'γₙ Index':<10} {'γₙ Value':<12} {'Dev %':<8}")
        print("-" * 60)
        
        for r in sorted(results, key=lambda x: x['deviation_pct']):
            print(f"{r['gift_constant']:<15} {r['target']:<8} γ_{r['nearest_index']:<7} {r['value']:<12.6f} {r['deviation_pct']:<8.4f}")
        
        return results

# Initialize analyzer
analyzer = GIFTRiemannAnalyzer(zeros_100k)

## 5. Run GIFT-Riemann Validation

In [ ]:
# Validate known correspondences
correspondences = analyzer.validate_gift_correspondences()

In [ ]:
# Test modified Pell equation
pell = analyzer.test_pell_equation()

In [ ]:
# Test recurrence relation
recurrence = analyzer.test_recurrence(n_test=10000)

In [ ]:
# Scan for all GIFT constants
scan = analyzer.scan_for_gift_constants(tolerance_pct=2.0)

## 6. High-Precision Computation with mpmath

In [ ]:
def compute_zeros_mpmath(start: int, count: int, precision: int = 50) -> list:
    """Compute zeros using mpmath (high precision but slow)."""
    if not MPMATH_AVAILABLE:
        raise ImportError("mpmath not available")
    
    mp.dps = precision
    zeros = []
    
    for n in tqdm(range(start, start + count), desc=f"Computing zeros (precision={precision})"):
        z = zetazero(n)
        zeros.append(float(z.imag))
    
    return zeros

# Example: compute first 10 zeros with 100 digits precision
if MPMATH_AVAILABLE:
    high_prec_zeros = compute_zeros_mpmath(1, 10, precision=100)
    print("\nHigh-precision zeros (100 digits):")
    for i, z in enumerate(high_prec_zeros, 1):
        print(f"  γ_{i} = {z}")

## 7. GPU-Accelerated Spectral Analysis

In [ ]:
def spectral_analysis_gpu(zeros: np.ndarray, max_freq: int = 1000):
    """GPU-accelerated FFT analysis of zero spacings."""
    if not GPU_AVAILABLE:
        print("GPU not available, using CPU")
        xp = np
    else:
        xp = cp
        zeros = cp.asarray(zeros)
    
    # Compute spacings
    spacings = xp.diff(zeros)
    
    # Normalize spacings
    mean_spacing = xp.mean(spacings)
    normalized = spacings / mean_spacing
    
    # FFT
    fft = xp.fft.fft(normalized)
    power = xp.abs(fft[:max_freq])**2
    freqs = xp.fft.fftfreq(len(normalized))[:max_freq]
    
    if GPU_AVAILABLE:
        power = power.get()
        freqs = freqs.get()
        cp.get_default_memory_pool().free_all_blocks()
    
    return freqs, power

# Run spectral analysis
freqs, power = spectral_analysis_gpu(zeros_100k)

# Find dominant frequencies
top_indices = np.argsort(power)[-10:][::-1]
print("\n=== Top 10 Spectral Peaks ===")
for i, idx in enumerate(top_indices):
    print(f"  {i+1}. freq={freqs[idx]:.6f}, power={power[idx]:.2f}")

## 8. Save Results

In [ ]:
def save_analysis_results(filename: str = "gift_riemann_analysis.json"):
    """Save all analysis results to JSON."""
    results = {
        'correspondences': correspondences,
        'pell_equation': pell,
        'recurrence': recurrence,
        'gift_constants_scan': [{k: (float(v) if isinstance(v, (np.floating, float)) else v) 
                                  for k, v in s.items()} for s in scan],
        'metadata': {
            'n_zeros': len(zeros_100k),
            'gpu_used': GPU_AVAILABLE,
            'source': 'Odlyzko zeros1'
        }
    }
    
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2, default=lambda x: float(x) if hasattr(x, 'item') else str(x))
    
    print(f"✓ Results saved to {filename}")
    return results

results = save_analysis_results()

## 9. Quick API Reference

```python
# Initialize cache and download data
cache = RiemannZeroCache()
zeros = cache.download_odlyzko("zeros1")  # First 100k
zeros = cache.download_odlyzko("zeros6")  # First 2M
zeros = cache.get_zeros(1, 1000)          # Get range by index

# Analyze with GPU
analyzer = GIFTRiemannAnalyzer(zeros, use_gpu=True)
analyzer.validate_gift_correspondences()
analyzer.test_pell_equation()
analyzer.test_recurrence(n_test=10000)
analyzer.scan_for_gift_constants(tolerance_pct=1.0)
analyzer.find_near_integer(target=14)     # Find zeros near any integer

# High-precision computation
zeros_hp = compute_zeros_mpmath(1, 100, precision=100)
```

In [ ]:
# Final status
cache.status()
print("\n✓ Notebook ready for GIFT-Riemann research!")